In [1]:
%pip install matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import plotly.express as px 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
from pathlib import Path
pd.set_option('display.max_columns', None)

Note: you may need to restart the kernel to use updated packages.


In [4]:
csv_path = "../data/raw_etf_data.csv"
stock_data = pd.read_csv(csv_path)

In [5]:
# Data Cleaning + Augmentation


stock_data['Date'] = pd.to_datetime(stock_data['Date'])
# Extract Year
stock_data['Year'] = stock_data['Date'].dt.year

# Create Month-Year column in YYYY-MM format
stock_data['Month_Year'] = stock_data['Date'].dt.to_period('M').astype(str)

# Extract Fiscal Quarter in YYYY-QX format
stock_data['Quarter'] = stock_data['Date'].dt.to_period('Q').astype(str)

# Extract ISO Week Number
stock_data['Week'] = stock_data['Date'].dt.isocalendar().week

# Daily return measures how much the stocks fluctuates since previous in percentages. Can be negative or positive. 
stock_data['Daily_Return'] = stock_data.groupby('Ticker')['Adj Close'].pct_change() * 100

# rolling max measures the highest the stock ever been, and updates dynamically. 
stock_data['Rolling_Max'] = stock_data.groupby('Ticker')['Adj Close'].cummax()

# 30 day volitility measures the STD from the average daily return of the last 30 day. The higher the STD, the more volitile. 
stock_data['Volatility_30d'] = stock_data.groupby('Ticker')['Daily_Return'].rolling(window=30).std().reset_index(0, drop=True)

# Compute drawdown as % drop from recent peak
stock_data['Drawdown'] = ((stock_data['Adj Close'] - stock_data['Rolling_Max']) / stock_data['Rolling_Max']) * 100

stock_data['Adj Close Normalized 0 - 1'] = stock_data.groupby('Ticker')['Adj Close'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min())
)
stock_data['Adj Close Normalized 0 - 100'] = stock_data['Adj Close Normalized 0 - 1'] * 100 

quarter_mapping = {
    'Q1': '-02-15',
    'Q2': '-05-15',
    'Q3': '-08-15',
    'Q4': '-11-15'
}

# Extract the year and quarter
stock_data['Year'] = stock_data['Quarter'].str[:4]   # Extracts '2000' from '2000Q1'
stock_data['Q'] = stock_data['Quarter'].str[4:]      # Extracts 'Q1' from '2000Q1'

# Create the new 'Mid_Quarter' column by mapping the quarter
stock_data['Mid_Quarter'] = stock_data['Year'] + stock_data['Q'].map(quarter_mapping)

# Convert to datetime format for proper sorting and plotting
stock_data['Mid_Quarter'] = pd.to_datetime(stock_data['Mid_Quarter'])

# Drop temporary columns if not needed
stock_data = stock_data.drop(columns=['Q'])

# Mapping for quarter-end dates
quarter_end_dates = {
    "Q1": "-03-31",
    "Q2": "-06-30",
    "Q3": "-09-30",
    "Q4": "-12-31"
}

# Extract the year and quarter
stock_data['Q'] = stock_data['Quarter'].str[4:]      # Extracts 'Q1' from '2000Q1'

# Create the new 'Mid_Quarter' column by mapping the quarter
stock_data['End_Quarter'] = stock_data['Year'] + stock_data['Q'].map(quarter_end_dates)
# Drop temporary columns if not needed
stock_data = stock_data.drop(columns=['Q'])

# Create a new columns to map ETF to correct industry

etf_industry_mapping = {
    "SPY": "Market Benchmark",
    "QQQ": "Technology",
    "XLK": "Technology",
    "XLE": "Energy",
    "TAN": "Energy",
    "XLV": "Healthcare",
    "IHI": "Healthcare",
    "XLF": "Financials",
    "KRE": "Financials",
    "VNQ": "Real Estate",
    "IYR": "Real Estate",
    "XLI": "Industrials",
    "ITA": "Industrials"
}
stock_data["Industry"] = stock_data["Ticker"].map(etf_industry_mapping)

In [6]:
# Read Fred Data 
macro_data = pd.read_csv('/Users/donnguyen/Desktop/industry-etf-performance/data/raw_macro_data.csv')

macro_indicators = {
    "DGS10": "10Y Treasury Yield",  # Interest rate, updates daily
    "CPIAUCSL": "Inflation (CPI)",  # Inflation, updates monthly 
    "GDP": "GDP Growth",  # Economic expansion/contraction, updates every 3 months (quarterly), by the billions
    "UNRATE": "Unemployment Rate",  # Job market indicator, Percentage of people actively looking for jobs but unemployed, update monthly 
    "FEDFUNDS": "Federal Funds Rate",  # Central bank borrowing cost, update 8 times per year. affects loans rate. Mortgage/auto/cc
    "VIXCLS": "VIX (Volatility Index)"  # Market volatility - The market's expected volatility over the next 30 days, updates daily
    # below 15 good above 30 bad. 
}

# Rename columns for readability
macro_data = macro_data.rename(columns=macro_indicators)

# Use ffill() to fill missing data since not all macro indicators update everyday. 
# Use bfill to fill data that ffill didn't capture. 
macro_data = macro_data.ffill().bfill()

# Calculate percentage changes for each indicator
macro_data['10Y Treasury Yield % Change'] = macro_data['10Y Treasury Yield'].pct_change() * 100
macro_data['Inflation Rate (%)'] = macro_data['Inflation (CPI)'].pct_change() * 100
macro_data['GDP Growth Rate %'] = macro_data['GDP Growth'].pct_change() * 100
macro_data['Δ Federal Funds Rate %'] = macro_data['Federal Funds Rate'].pct_change() * 100
macro_data['Δ VIX %'] = macro_data['VIX (Volatility Index)'].pct_change() * 100
macro_data = macro_data.reset_index()

# rename 
macro_data = macro_data.rename(columns={"DATE": "Date"})
macro_data["Date"] = pd.to_datetime(macro_data["Date"])


In [7]:
# Merge the Macro Data with the ETF data 
df = stock_data.merge(macro_data, on="Date", how="left")

In [8]:
# Rearrange column order for better readability 

columns = [
    # Time Features
    'Date', 'Year', 'Month_Year', 'Quarter', 'Week', 'Mid_Quarter', 'End_Quarter',
    
    # Stock Info
    'Ticker', "Industry",
    
    # Price & Market Data
    'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume',
    
    # Stock Performance Metrics
    'Daily_Return', 'Rolling_Max', 'Volatility_30d', 'Drawdown',
    
    # Normalized Price Data
    'Adj Close Normalized 0 - 1', 'Adj Close Normalized 0 - 100',

    # Macroeconomic Indicators
    '10Y Treasury Yield', 'Inflation (CPI)', 'GDP Growth', 
    'Unemployment Rate', 'Federal Funds Rate', 'VIX (Volatility Index)',
    
    # Macro % Changes
    '10Y Treasury Yield % Change', 'Inflation Rate (%)', 'GDP Growth Rate %', 
    'Δ Federal Funds Rate %', 'Δ VIX %'
]
df = df[columns]
